In [ ]:
import os
import io
import csv
from read_data import convert_pdf_to_txt
from PIL import Image
import pytesseract
from wand.image import Image as wi
import hashlib
import pandas as pd
import numpy as np
import pprint as pp
from urllib.request import urlopen
import time

In [ ]:
def get_size(st):
    if os.path.isfile(st):
        return os.path.getsize(st)

In [ ]:
def conv_pdf(pdf):
    if os.path.isfile('./pdfs/pdfs/' + pdf):
        print('--> ' + pdf)
        with open('./pdfs/pdfs/' + pdf, 'rb') as f:
            # text from pdf
            bfr = io.BufferedReader(f)
            pdf_text = convert_pdf_to_txt(bfr) 
            
            # recognized text (OCR) from pdf
            if len(pdf_text.strip()) <= 50:
                pdf_file = wi(filename='./pdfs/pdfs/' + pdf, resolution=200)
                pdfImage = pdf_file.convert('jpeg')
                imageBlobs = []
                for img in pdfImage.sequence:
                    imgPage = wi(image=img)
                    imageBlobs.append(imgPage.make_blob('jpeg'))

                recognized_text = []

                for imgBlob in imageBlobs:
                    im = Image.open(io.BytesIO(imgBlob))
                    text = pytesseract.image_to_string(im, lang = 'eng')
                    recognized_text.append(text)

                recognized_text = '\n\n\n'.join(recognized_text)

            pdf_text = pdf_text if len(pdf_text.strip()) > 50 else recognized_text
            
            return pdf_text

In [ ]:
oregon_csv = pd.read_csv('oregon_20170904.csv', index_col = 1) # change the list len

In [ ]:
oregon_csv['size'] = oregon_csv.filename.apply(lambda x: './pdfs/pdfs/' + x).apply(get_size)
oregon_csv = oregon_csv.sort_values('size')

In [ ]:
oregon_csv = oregon_csv[30590:30600]

In [ ]:
oregon_csv = oregon_csv[oregon_csv.filename != 'CommitteeMeetingDocument_121140.pdf'] 
oregon_csv = oregon_csv[oregon_csv.filename != 'CommitteeMeetingDocument_119635.pdf']
oregon_csv = oregon_csv[oregon_csv.filename != 'CommitteeMeetingDocument_110713.pdf']
oregon_csv = oregon_csv[oregon_csv.filename != 'CommitteeMeetingDocument_135717.pdf']
# oregon_csv = oregon_csv[oregon_csv.filename != 'CommitteeMeetingDocument_118799.pdf']
oregon_csv.describe(include='all')

In [ ]:
oregon_csv['text'] = oregon_csv.filename.apply(conv_pdf)

In [ ]:
oregon_csv

In [ ]:
oregon_csv.loc[:, ['bill', 'session', 'text', 'url', 'filename']].to_csv('oregon-pt63.csv')

In [ ]:
fil_csv = pd.read_csv('oregon1001-1890.csv', index_col=0)

In [ ]:
b = fil_csv[fil_csv[['text']].isnull().values]

In [ ]:
b.loc[:, 'filename'] = b.url.apply(write_pdf)

In [ ]:
b['text'] = b.filename.apply(conv_pdf)

In [ ]:
b

In [ ]:
b.to_csv('oregon_missed_115.csv')

In [ ]:
b.loc[:, 'filename'] = fil_csv[fil_csv[['text']].isnull().values].url.apply(write_pdf)

In [ ]:
b['text'] = b.filename.apply(conv_pdf)

In [ ]:
b

In [ ]:
b.to_csv('oregon_missed_20.csv')